# Projet Datascientest - Compagnon immo

<code>mar25_bds_compagnon_immo_1</code>

---

## Exploration de données - v1.0

**Objectifs :**
- Définir le contexte et le périmètre du projet, compréhension du projet
- Découverte du jeu de données, mise en lumière la structure, les difficultés et eventuels biais du dataset.

---

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import os
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO
import gzip

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

---

### **Analyse du dataset sur les ventes immobilières**



---

In [57]:
url_sales = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/ech_annonces_ventes_68.csv'

sales_df = pd.read_csv(url_sales, sep=';', index_col="idannonce")

display(sales_df.head())

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
immo-facile-52471952,pr,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,1512.83
immo-facile-52477215,pr,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,NaN,NaN,True,0,G,NaN,1.0,1.0,NaN,NaN,NaN,False,2.0,180.0,Fioul,Radiateur,Collectif,a,n,8.0,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,7.93,3.0,0.092,7.93,3.0,0.092,1039.47
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22


In [5]:
dataset_info = pd.DataFrame({
    'Value': [sales_df.shape[0], 
              sales_df.shape[1], 
              sales_df.duplicated().sum(), 
              round((sales_df.isna().sum().sum() / sales_df.size) * 100, 2),
              len(sales_df.select_dtypes('number').columns),
              len(sales_df.select_dtypes('object').columns)
             ]
}, index=['nombre de lignes', 
          'nombre de colonnes', 
          'nombre de doublons', 
          'manquants (%)', 
          'nombre de variables quantitatives', 
          'nombre de variables qualitatives'])
dataset_info

,Value
nombre de lignes,27737.00
nombre de colonnes,58.00
nombre de doublons,0.00
manquants (%),28.68
nombre de variables quantitatives,37.00
nombre de variables qualitatives,21.00


In [7]:
dtype_info = pd.DataFrame({
    'colonne': sales_df.columns,
    'type': sales_df.dtypes
}).set_index('colonne')

dtype_info

,type
colonne,
type_annonceur,object
typedebien,object
typedetransaction,object
etage,int64
surface,int64
surface_terrain,float64
nb_pieces,int64
prix_bien,int64
prix_maison,float64


---
> **Valeurs manquantes**

In [217]:
missing_percent = (sales_df.isnull().sum() / len(sales_df)) * 100

missing_df = pd.DataFrame({
    'colonne': sales_df.columns,
    'manquants (%)': round(missing_percent,2)
}).set_index('colonne')

display(missing_df.sort_values(by='manquants (%)', ascending=False))

,manquants (%)
colonne,
parking,99.90
videophone,99.51
prix_maison,98.42
prix_terrain,98.42
surface_balcon,97.63
nb_terraces,85.22
porte_digicode,84.63
charges_copro,75.93
nb_logements_copro,72.73


---
> **Modalités des valeurs qualitatives**

In [7]:
cat_columns = sales_df.select_dtypes('object')
cat_modes = []
for col in cat_columns:
    modalities = ', '.join(sales_df[col].unique().astype(str))
    cat_modes.append({'colonne' : col, 'modalités': modalities, 'nombre modalités': sales_df[col].nunique()})

modalities_df = pd.DataFrame(cat_modes).set_index('colonne')
modalities_df

,modalités,nombre modalités
colonne,,
type_annonceur,pr,1
typedebien,"a, m, an, mn, l",5
typedetransaction,"v, vp, pi",3
dpeL,"C, 0, E, G, NS, D, F, VI, A, B",10
annonce_exclusive,"Non, 0, Oui",3
parking,"nan, True, False",2
cave,"False, nan, True",2
exposition,"0, Sud, Ouest, Nord-Ouest, Sud-Est, traversant...",35
ges_class,"C, nan, A, G, B, E, F, D, VI, NS",9


---

> **Modalités des valeurs quantitatives**

In [7]:
sales_df.describe().round(2)

,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,nb_etages,places_parking,annee_construction,nb_toilettes,nb_terraces,surface_balcon,nb_logements_copro,charges_copro,duree_int,INSEE_COM,IRIS,CODE_IRIS,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
count,27737.00,27737.00,11444.00,27737.00,27737.00,437.00,437.00,27737.00,27737.00,27737.00,27737.00,17150.00,27737.00,27737.00,14859.00,10169.00,9458.00,15436.00,4099.00,658.00,7565.00,6677.00,26196.00,27737.00,27737.00,2.773700e+04,27737.00,27737.00,27737.0,27737.0,17485.00,17485.00,17485.00,19650.00,19650.00,19650.00,27737.00
mean,1.39,108.34,1017.41,4.53,277342.67,213134.46,89757.58,13.23,0.17,0.28,0.68,205.78,47.81,7.34,2.87,1.98,1970.00,1.49,0.99,12.49,53.63,1805.80,-118.18,68193.43,172.92,6.819345e+08,6819343.68,68370.91,44.0,68.0,10.78,17.01,0.06,10.64,151.04,0.06,2624.97
std,26.85,60.85,6694.22,2.03,186135.23,71437.52,47633.30,139.08,0.42,0.54,0.72,102.92,0.18,0.13,2.59,1.60,56.27,0.75,0.57,32.20,86.45,31551.74,390.37,106.25,336.89,1.062528e+06,10625.42,269.82,0.0,0.0,2.07,29.39,0.08,1.95,252.45,0.08,1195.39
min,-1.00,2.00,1.00,1.00,1076.00,86000.00,1.00,0.00,0.00,0.00,0.00,0.00,47.43,6.86,0.00,0.00,971.00,0.00,0.00,1.00,0.00,0.00,-1125.00,68001.00,0.00,6.800100e+08,6800100.00,68000.00,44.0,68.0,3.08,1.00,0.00,4.78,1.00,0.00,15.59
25%,0.00,71.00,378.00,3.00,159000.00,156500.00,62000.00,0.00,0.00,0.00,0.00,135.00,47.67,7.27,2.00,1.00,1950.00,1.00,1.00,6.00,10.00,660.00,6.00,68084.00,0.00,6.808400e+08,6808400.00,68112.00,44.0,68.0,9.44,2.00,0.04,9.32,3.00,0.04,1870.00
50%,0.00,95.00,589.00,4.00,242000.00,199620.00,87000.00,0.00,0.00,0.00,1.00,196.00,47.76,7.34,2.00,2.00,1978.00,1.00,1.00,10.00,24.00,1200.00,22.00,68222.00,101.00,6.822200e+08,6822200.00,68403.00,44.0,68.0,10.53,6.00,0.05,10.38,18.00,0.05,2580.65
75%,1.00,130.00,875.00,5.00,347000.00,250010.00,115800.00,0.00,0.00,1.00,1.00,262.00,47.93,7.41,3.00,2.00,2007.00,2.00,1.00,13.00,66.00,2000.00,77.00,68279.00,104.00,6.827900e+08,6827900.00,68701.00,44.0,68.0,11.95,19.00,0.07,11.81,190.00,0.07,3268.48
max,999.00,980.00,431600.00,43.00,3380000.00,514999.00,320000.00,7715.00,8.00,10.00,12.00,988.00,48.30,7.62,36.00,35.00,2025.00,15.00,14.00,801.00,3442.00,2577352.00,793.00,68386.00,1802.00,6.838600e+08,6838600.00,68701.00,44.0,68.0,23.73,212.00,10.50,23.73,960.00,10.77,79500.00


In [9]:
num_columns = sales_df.select_dtypes('number')
num_modes = []
for col in num_columns:
    modalities = ', '.join(sales_df[col].unique().astype(str))
    num_modes.append({'colonne' : col, 'modalités': modalities, 'nombre modalités': sales_df[col].nunique()})

modalities_df = pd.DataFrame(num_modes).set_index('colonne')
modalities_df

,modalités,nombre modalités
colonne,,
etage,"0, 5, 8, 1, 6, 4, 2, 3, 7, 21, 11, 9, 999, 10,...",24
surface,"64, 85, 187, 76, 90, 82, 134, 114, 69, 154, 87...",398
surface_terrain,"nan, 4950.0, 3182.0, 3071.0, 872.0, 11140.0, 3...",2097
nb_pieces,"3, 4, 9, 5, 6, 2, 1, 7, 20, 10, 8, 19, 11, 14,...",22
prix_bien,"128400, 189000, 282900, 79000, 317000, 170000,...",5069
prix_maison,"nan, 261380.0, 308880.0, 200500.0, 233700.0, 2...",313
prix_terrain,"nan, 158620.0, 136000.0, 98000.0, 115800.0, 14...",211
mensualiteFinance,"0, 248, 790, 666, 553, 378, 1311, 313, 359, 19...",325
balcon,"1, 0, 2, 4, 3, 8",6


> **Répartition temporelle**

In [70]:
sales_df['year'] = pd.to_datetime(sales_df['date']).dt.year
sales_df.groupby('year').size()

year
2019     722
2020    6610
2021    7041
2022    9399
2023    3965
dtype: int64

> **Recherches informations outliers**

In [9]:
# Etage
col = 'etage'
print("\n---------------------- Etage ---------------------")
print(f"Etage minimal                         : {sales_df[col].min()}")
print(f"Nombre etage minimal                  : {len(sales_df.loc[sales_df[col] == sales_df[col].min()])}")
print(f"Etage maximal                         : {sales_df[col].max()}")
print(f"Nombre etage maximal                  : {len(sales_df.loc[sales_df[col] == sales_df[col].max()])}")
print("-" * 50)
print("Ces valeurs ne semblent concernés que les appartements")
print("Pour les etages -1 une hypothèse pour des souplex ou sous-sols aménagés, la décote varie entre 15 et 30 % selon l’emplacement")
print("Pour les etages 999, cela semble être une valeur par défaut.")
print("\n")
display(sales_df.loc[sales_df[col] == -1, [
    col, 'surface', 'prix_m2_vente', 'typedebien', 'balcon', 'cave']])

display(sales_df.loc[sales_df[col] == 999, [
    col, 'surface', 'prix_m2_vente', 'typedebien', 'GRD_QUART', 'mapCoordonneesLatitude', 'mapCoordonneesLongitude']])


---------------------- Etage ---------------------
Etage minimal                         : -1
Nombre etage minimal                  : 7
Etage maximal                         : 999
Nombre etage maximal                  : 20
--------------------------------------------------
Ces valeurs ne semblent concernés que les appartements
Pour les etages -1 une hypothèse pour des souplex ou sous-sols aménagés, la décote varie entre 15 et 30 % selon l’emplacement
Pour les etages 999, cela semble être une valeur par défaut.




,etage,surface,prix_m2_vente,typedebien,balcon,cave
idannonce,,,,,,
immo-facile-50836736,-1,140,984.29,a,0,True
ag671474-343036833,-1,71,2971.83,a,1,False
hektor-moebel-immo-126,-1,110,2963.64,a,0,False
hektor-lifeimmo-1934,-1,48,1625.00,a,0,False
hektor-MAYAIMMO2020-2055,-1,75,2798.67,a,0,True
hektor-lifeimmo-1904,-1,45,2422.22,a,0,False
ag750949-297651000,-1,28,1857.14,a,1,False


,etage,surface,prix_m2_vente,typedebien,GRD_QUART,mapCoordonneesLatitude,mapCoordonneesLongitude
idannonce,,,,,,,
apimo-8213324,999,62,2090.32,a,6806601,48.08927,7.36480
apimo-8126551,999,76,2236.84,a,6806601,48.07967,7.36152
apimo-7906195,999,50,2090.00,a,6806601,48.08553,7.32811
apimo-7651264,999,76,2362.50,a,6804200,47.60663,7.48502
apimo-7498512,999,66,2575.76,a,6806601,48.07747,7.35446
apimo-7461643,999,56,5500.00,a,6806601,48.07517,7.37945
apimo-7461355,999,108,5453.70,a,6806601,48.07517,7.37945
apimo-7267452,999,88,2409.09,a,6806601,48.06558,7.36342
apimo-6958660,999,99,3030.30,a,6828700,47.95986,7.30080


In [270]:
# Année construction
quartier_code = 6825700
col = "annee_construction"
print("\n-------------- Année de construction -------------")
print(f"Année minimale                          : {sales_df[col].min()}")
print(f"Nombre d'années < 1000                  : {len(sales_df.loc[sales_df[col] < 1000])}")
print(f"Nombre d'années proche 1970 du quartier : {len(sales_df[(sales_df[col] >= 1970) & 
                                                      (sales_df[col] < 1980) &
                                                      (sales_df['GRD_QUART'] == quartier_code)])}")
print(f"Médiane du code quartier '{quartier_code}'      : {sales_df.loc[sales_df['GRD_QUART'] == quartier_code][col].median()}")
print("-" * 50)
print("Cas isolé, semble être une erreur de saisie avec 1971")
print("\n")



-------------- Année de construction -------------
Année minimale                          : 971.0
Nombre d'années < 1000                  : 1
Nombre d'années proche 1970 du quartier : 2
Médiane du code quartier '6825700'      : 1920.0
--------------------------------------------------
Cas isolé, semble être une erreur de saisie avec 1971




In [272]:
# nb_logements_copro
col = 'nb_logements_copro'
print("\n--------------- Nbre logements copro -------------")
print(f"Nbre logements copro médian              : {sales_df[col].median()}")
print(f"Nbre logements copro maximal             : {sales_df[col].max()}")
print(f"Nbre logements copro > 1000              : {len(sales_df.loc[sales_df[col] > 1000])}")
print("-" * 50)
print("Le 1er cas représente une zone spécifique")
print("Le 2eme cas représente un bâtiment de 19 étages.")
print("\n")
sales_df.loc[sales_df[col] > 1000, [col, 'etage', 'TYP_IRIS_x', 'TYP_IRIS_y']]



--------------- Nbre logements copro -------------
Nbre logements copro médian              : 24.0
Nbre logements copro maximal             : 3442.0
Nbre logements copro > 1000              : 2
--------------------------------------------------
Le 1er cas représente une zone spécifique
Le 2eme cas représente un bâtiment de 19 étages.




,nb_logements_copro,etage,TYP_IRIS_x,TYP_IRIS_y
idannonce,,,,
guy-hoquet-immo-facile-5612825,1001.0,0,Z,Z
safti-1-486917,3442.0,19,H,H


In [274]:
# charges_copro
col = 'charges_copro'
print("\n------------------- Charges copro ----------------")
print(f"Nbre Charges copro  médian              : {sales_df[col].median()}")
print(f"Nbre Charges copro  maximal             : {sales_df[col].max()}")
print(f"Nbre Charges copro > 100000             : {len(sales_df.loc[sales_df[col] > 100000])}")
print("-" * 50)
print("Cas isolé, semble être une erreur de saisie ou une erreur de conversion")
print("\n")
sales_df.loc[sales_df[col] > 100000, ['typedebien', 'nb_etages', 'nb_logements_copro', col]]


------------------- Charges copro ----------------
Nbre Charges copro  médian              : 1200.0
Nbre Charges copro  maximal             : 2577352.0
Nbre Charges copro > 100000             : 1
--------------------------------------------------
Cas isolé, semble être une erreur de saisie ou une erreur de conversion




,typedebien,nb_etages,nb_logements_copro,charges_copro
idannonce,,,,
ag671474-282219599,a,4.0,180.0,2577352.0


In [282]:
# dpeC
seuil = 800
col = 'dpeC'
print("\n-------------------- DPE C ------------------")
print(f"Nbre DPE C  médian           : {sales_df[col].median()}")
print(f"Nbre DPE C  maximal          : {sales_df[col].max()}")
print(f"Nbre DPE C > {seuil}             : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 45)
print("Ceux sont des valeurs extrêmes et non aberrantes car la performance énergétique 'dpeL' est mauvaise également")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'dpeC', 'dpeL', 'ges_class', 'typedebien', 'chauffage_energie', 'chauffage_systeme']]


-------------------- DPE C ------------------
Nbre DPE C  médian           : 196.0
Nbre DPE C  maximal          : 988.0
Nbre DPE C > 800             : 8
---------------------------------------------
Ceux sont des valeurs extrêmes et non aberrantes car la performance énergétique 'dpeL' est mauvaise également




,surface,dpeC,dpeL,ges_class,typedebien,chauffage_energie,chauffage_systeme
idannonce,,,,,,,
ag681391-378826546,154,935.0,G,G,m,NaN,NaN
ag681088-367567792,80,849.0,G,C,m,NaN,NaN
ag681237-349472634,127,930.0,G,C,m,NaN,NaN
ag681391-336113367,340,819.0,G,F,m,NaN,NaN
hektor-MAYAIMMO2020-2199,23,988.0,G,D,a,Électrique,Radiateur
immo-facile-46134166,75,840.0,G,F,m,Électrique,NaN
orpi-1-054927E1JM3Y,47,972.0,G,NaN,a,NaN,NaN
hektor-robischung-2263,155,831.0,G,D,m,Fioul,Radiateur


In [288]:
# nb_toilettes
seuil = 10
col = 'nb_toilettes'
print("\n------------------ Nbre toilettes ----------------")
print(f"Nbre toilettes  médian           : {sales_df[col].median()}")
print(f"Nbre toilettes  maximal          : {sales_df[col].max()}")
print(f"Nbre toilettes > {seuil}              : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 50)
print("Le type IRIS représente une zone spécifique (biens avec sanitaires collectifs ou autres ?)")
print("\n")
sales_df.loc[sales_df[col] > seuil, [col, 'surface', 'typedebien', 'nb_pieces', 'TYP_IRIS_x', 'TYP_IRIS_y']]


------------------ Nbre toilettes ----------------
Nbre toilettes  médian           : 1.0
Nbre toilettes  maximal          : 15.0
Nbre toilettes > 10              : 4
--------------------------------------------------
Le type IRIS représente une zone spécifique (biens avec sanitaires collectifs ou autres ?)




,nb_toilettes,surface,typedebien,nb_pieces,TYP_IRIS_x,TYP_IRIS_y
idannonce,,,,,,
ag681237-376181851,15.0,710,m,15,Z,Z
ag671717-371144704,11.0,43,a,2,Z,Z
safti-1-763905,11.0,70,m,3,Z,Z
hektor-1373403-213,13.0,534,m,15,Z,Z


In [11]:
# nb_terraces
seuil = 10
col = 'nb_terraces'
print("\n------------------ Nbre terrasses ----------------")
print(f"Nbre terrasses  médian           : {sales_df[col].median()}")
print(f"Nbre terrasses  maximal          : {sales_df[col].max()}")
print(f"Nbre terrasses > {seuil}              : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 50)
print("Cas isolé, le nombre de pièce à vivre est cohérent avec le nombre de terrasse")
print("\n")
sales_df.loc[sales_df[col] > seuil, [col, 'surface', 'typedebien', 'nb_pieces', 'TYP_IRIS_x', 'TYP_IRIS_y']]


------------------ Nbre terrasses ----------------
Nbre terrasses  médian           : 1.0
Nbre terrasses  maximal          : 14.0
Nbre terrasses > 10              : 1
--------------------------------------------------
Cas isolé, le nombre de pièce à vivre est cohérent avec le nombre de terrasse




,nb_terraces,surface,typedebien,nb_pieces,TYP_IRIS_x,TYP_IRIS_y
idannonce,,,,,,
hektor-MAYAIMMO2020-1827,14.0,206,m,7,Z,Z


In [310]:
# places_parking
seuil = 20
col = 'places_parking'
print("\n------------- Nbre places parking -----------")
print(f"Nbre places parking médian       : {sales_df[col].median()}")
print(f"Nbre places parking maximal      : {sales_df[col].max()}")
print(f"Nbre places parking > {seuil}         : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 45)
print("En règle générale, le nombre important de places de parking se trouvent dans des maisons luxueuses.")
print("Le dernier cas ne semble pas justifié car le type de bien est un appartement.")
print("\n")
sales_df.loc[sales_df[col] > seuil, [col, 'surface', 'typedebien', 'nb_pieces', 'TYP_IRIS_x', 'TYP_IRIS_y', 'prix_bien']]


------------- Nbre places parking -----------
Nbre places parking médian       : 2.0
Nbre places parking maximal      : 35.0
Nbre places parking > 20         : 3
---------------------------------------------
En règle générale, le nombre important de places de parking se trouvent dans des maisons luxueuses.
Le dernier cas ne semble pas justifié car le type de bien est un appartement.




,places_parking,surface,typedebien,nb_pieces,TYP_IRIS_x,TYP_IRIS_y,prix_bien
idannonce,,,,,,,
hektor-adiflo-1671,30.0,500,m,10,H,H,3161000
ag340369-346403481,35.0,253,m,10,H,H,420000
orpi-1-054927E26B7K,30.0,73,a,3,H,H,233700


In [320]:
# nb_pieces
seuil = 20
col = 'nb_pieces'
print("\n----------------- Nbre pieces ---------------")
print(f"Nbre pieces médian           : {sales_df[col].median()}")
print(f"Nbre pieces maximal          : {sales_df[col].max()}")
print(f"Nbre pieces > {seuil}             : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 45)
print("La 2eme ligne ressemble à une erreur de saisie, la surface n'est pas cohérente.")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'typedebien', col, 'TYP_IRIS_x', 'TYP_IRIS_y', 'prix_m2_vente', 'prix_bien']]


----------------- Nbre pieces ---------------
Nbre pieces médian           : 4.0
Nbre pieces maximal          : 43
Nbre pieces > 20             : 2
---------------------------------------------
La 2eme ligne ressemble à une erreur de saisie, la surface n'est pas cohérente.




,surface,typedebien,nb_pieces,TYP_IRIS_x,TYP_IRIS_y,prix_m2_vente,prix_bien
idannonce,,,,,,,
157949837,883,m,22,Z,Z,475.65,420000
146173003,2,an,43,H,H,79500.00,159000


In [322]:
# surface_balcon
col = 'surface_balcon'
print("\n------------------ surface balcon ----------------")
print(f"surface balcon médian         : {sales_df[col].median()}")
print(f"surface balcon maximal        : {sales_df[col].max()}")
print(f"surface balcon > surface      : {len(sales_df.loc[sales_df[col] > sales_df['surface']])}")
print("-" * 50)
print("Valeur aberrante, la surface balcon est > à la surface du bien")
print("\n")
sales_df.loc[sales_df[col] > sales_df['surface'], ['surface', col, 'typedebien', 'nb_pieces', 'prix_bien']]


------------------ surface balcon ----------------
surface balcon médian         : 10.0
surface balcon maximal        : 801.0
surface balcon > surface      : 1
--------------------------------------------------
Valeur aberrante, la surface balcon est > à la surface du bien




,surface,surface_balcon,typedebien,nb_pieces,prix_bien
idannonce,,,,,
hektor-MAYAIMMO2020-2308,88,801.0,a,4,140000


In [330]:
# surface
col = 'surface'
seuil = 800
print("\n------------------- surface -----------------")
print(f"surface médian         : {sales_df[col].median()}")
print(f"surface maximal        : {sales_df[col].max()}")
print(f"surface > {seuil}m²        : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 45)
print("Difficile d'avoir une interprétation pour les biens dans des zones spécifiques (Z)")
print("\n")
sales_df.loc[sales_df[col] > seuil, [col, 'TYP_IRIS_x', 'TYP_IRIS_y', 'typedebien', 'nb_pieces', 'prix_bien']]


------------------- surface -----------------
surface médian         : 95.0
surface maximal        : 980
surface > 800m²        : 5
---------------------------------------------
Difficile d'avoir une interprétation pour les biens dans des zones spécifiques (Z)




,surface,TYP_IRIS_x,TYP_IRIS_y,typedebien,nb_pieces,prix_bien
idannonce,,,,,,
century-21-202_3377_564,980,H,H,m,13,1155000
160967155,835,Z,Z,mn,5,386100
ag670547-252666901,825,Z,Z,m,10,835000
157949837,883,Z,Z,m,22,420000
153896499,876,Z,Z,a,4,231200


In [338]:
# surface_terrain
seuil = 100000
col = 'surface_terrain'
print("\n----------------- surface terrain ----------------")
print(f"surface terrain médian         : {sales_df[col].median()}")
print(f"surface terrain maximal        : {sales_df[col].max()}")
print(f"surface terrain > {seuil}m²     : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 50)
print("Pas d'hypothèse d'abberation ici, il peut s'agir de bois ou de terrain agricole dépendant du bien")
print("Pour la plus grande valeur Google Map nous montre l'emplacement d'un bois.")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', col, 'typedebien', 'nb_pieces', 'prix_bien', 'mapCoordonneesLatitude', 'mapCoordonneesLongitude']]


----------------- surface terrain ----------------
surface terrain médian         : 589.0
surface terrain maximal        : 431600.0
surface terrain > 100000m²     : 7
--------------------------------------------------
Pas d'hypothèse d'abberation ici, il peut s'agir de bois ou de terrain agricole dépendant du bien
Pour la plus grande valeur Google Map nous montre l'emplacement d'un bois.




,surface,surface_terrain,typedebien,nb_pieces,prix_bien,mapCoordonneesLatitude,mapCoordonneesLongitude
idannonce,,,,,,,
adapt-immo-680091297,232,250000.0,m,7,910000,47.58936,7.46701
apimo-6722595,720,350000.0,m,11,1800000,47.84972,7.21767
ag681109-328163280,76,102600.0,m,3,283000,47.65633,7.45660
ag340369-305226948,130,170000.0,m,5,241500,47.89031,6.99813
adapt-immo-680091350,330,431600.0,m,12,262500,47.48255,7.33706
adapt-immo-680091479,160,170000.0,m,6,416000,47.55317,7.37481
adapt-immo-680091433,110,120000.0,m,6,364000,47.55317,7.37481


In [340]:
# prix_bien
seuil = 3000000
col = 'prix_bien'
print("\n---------------------- prix bien ---------------------")
print(f"prix bien médian                            : {sales_df[col].median()}")
print(f"prix bien maximal                           : {sales_df[col].max()}")
print(f"prix bien > {seuil}                         : {len(sales_df.loc[sales_df[col] > seuil])}")
print(f"prix bien > 1 000 000 avec surface < 100 m² : {len(sales_df.loc[(sales_df['prix_bien'] > 1000000) & (sales_df['prix_bien'] < 100)])}")
print("-" * 55)
print("Les prix sont cohérents avec le nombre de pièce et la surface.")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'surface_terrain', 'typedebien', 'nb_pieces', 'prix_bien']]


---------------------- prix bien ---------------------
prix bien médian                            : 242000.0
prix bien maximal                           : 3380000
prix bien > 3000000                         : 3
prix bien > 1 000 000 avec surface < 100 m² : 0
-------------------------------------------------------
Les prix sont cohérents avec le nombre de pièce et la surface.




,surface,surface_terrain,typedebien,nb_pieces,prix_bien
idannonce,,,,,
ag681391-375978205,467,1876.0,m,11,3380000
ag681391-372166195,467,1876.0,m,11,3380000
hektor-adiflo-1671,500,3362.0,m,10,3161000


In [348]:
# prix_maison
seuil = 400000
col = 'prix_maison'
print("\n---------------------- prix maison ---------------------")
print(f"prix maison médian                          : {sales_df[col].median()}")
print(f"prix maison maximal                         : {sales_df[col].max()}")
print(f"prix maison > {seuil}                        : {len(sales_df.loc[sales_df[col] > seuil])}")
print(f"prix maison > 100 000 avec type appartement : {len(sales_df.loc[(sales_df[col] > 100000) & (sales_df['typedebien'] == 'a')])}")
print("-" * 55)
print("Pas de valeurs aberrantes")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'surface_terrain', 'typedebien', 'nb_pieces', 'prix_bien', 'prix_maison']]


---------------------- prix maison ---------------------
prix maison médian                          : 199620.0
prix maison maximal                         : 514999.0
prix maison > 400000                        : 7
prix maison > 100 000 avec type appartement : 0
-------------------------------------------------------
Pas de valeurs aberrantes




,surface,surface_terrain,typedebien,nb_pieces,prix_bien,prix_maison
idannonce,,,,,,
160770041,190,800.0,mn,4,730000,410000.0
160935619,97,853.0,mn,4,515000,514999.0
161436223,132,558.0,mn,4,412000,411999.0
160935617,170,1.0,mn,4,472900,472100.0
160878351,138,852.0,mn,5,454420,454419.0
159636389,138,601.0,mn,5,415600,415599.0
160423293,140,500.0,mn,5,475000,474999.0


In [352]:
# prix_terrain
seuil = 200000
col = 'prix_terrain'
print("\n-------------------- prix terrain ---------------------")
print(f"prix terrain médian                          : {sales_df[col].median()}")
print(f"prix terrain maximal                         : {sales_df[col].max()}")
print(f"prix terrain > {seuil}                        : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 55)
print("Pas de valeurs aberrantes")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'surface_terrain', 'typedebien', 'nb_pieces', 'prix_bien', 'prix_maison', 'prix_terrain']]


-------------------- prix terrain ---------------------
prix terrain médian                          : 87000.0
prix terrain maximal                         : 320000.0
prix terrain > 200000                        : 4
-------------------------------------------------------
Pas de valeurs aberrantes




,surface,surface_terrain,typedebien,nb_pieces,prix_bien,prix_maison,prix_terrain
idannonce,,,,,,,
160770041,190,800.0,mn,4,730000,410000.0,320000.0
160555523,120,556.0,mn,5,564210,319570.0,244640.0
160555513,138,556.0,mn,4,574550,329910.0,244640.0
158957333,140,1036.0,mn,5,519000,248360.0,270640.0


In [354]:
# mensualiteFinance
seuil = 4000
col = 'mensualiteFinance'
print("\n------------------ mensualite Finance -----------------")
print(f"mensualiteFinance médian                       : {sales_df[col].median()}")
print(f"mensualiteFinance maximal                      : {sales_df[col].max()}")
print(f"mensualiteFinance > {seuil}                       : {len(sales_df.loc[sales_df[col] > seuil])}")
print("-" * 55)
print("Pas de valeurs aberrantes, les types de transactions concernent des ventes en cours")
print("\n")
sales_df.loc[sales_df[col] > seuil, ['surface', 'typedebien', 'typedetransaction', 'prix_bien', 'prix_maison', 'mensualiteFinance']]


------------------ mensualite Finance -----------------
mensualiteFinance médian                       : 0.0
mensualiteFinance maximal                      : 7715
mensualiteFinance > 4000                       : 2
-------------------------------------------------------
Pas de valeurs aberrantes, les types de transactions concernent des ventes en cours




,surface,typedebien,typedetransaction,prix_bien,prix_maison,mensualiteFinance
idannonce,,,,,,
153296215,400,m,vp,1870000,NaN,7715
153060803,170,m,vp,1290000,NaN,5325


In [358]:
# prix_m2_vente
seuil_min = 100
seuil_max = 10000
col = 'prix_m2_vente'
print("\n---------------------- prix m2 vente ---------------------")
print(f"prix m2 vente minimal                      : {sales_df[col].min()}")
print(f"prix m2 vente médian                       : {sales_df[col].median()}")
print(f"prix m2 vente maximal                      : {sales_df[col].max()}")
print(f"prix m2 vente < {seuil_min}                        : {len(sales_df.loc[sales_df[col] < seuil_min])}")
print(f"prix m2 vente > {seuil_max}                      : {len(sales_df.loc[sales_df[col] > seuil_max])}")
print(f"prix m2 vente > {seuil_max} et surface < 50 m²   : {len(sales_df.loc[(sales_df[col] > seuil_max) & (sales_df['surface'] < 50)])}")
print("-" * 55)
print("Les surfaces sont anormalement petites pour le prix élevé au m²")
print("\n")
sales_df.loc[
    (sales_df[col] > seuil_max) & (sales_df['surface'] < 50),
    ['surface', 'typedebien', 'nb_pieces', 'prix_bien', 'prix_maison', 'prix_m2_vente']
]


---------------------- prix m2 vente ---------------------
prix m2 vente minimal                      : 15.59
prix m2 vente médian                       : 2580.65
prix m2 vente maximal                      : 79500.0
prix m2 vente < 100                        : 2
prix m2 vente > 10000                      : 8
prix m2 vente > 10000 et surface < 50 m²   : 6
-------------------------------------------------------
Les surfaces sont anormalement petites pour le prix élevé au m²




,surface,typedebien,nb_pieces,prix_bien,prix_maison,prix_m2_vente
idannonce,,,,,,
mgc-379653425-CWE-230321-155600,8,m,5,214000,NaN,26750.00
gedeon-24739060,17,a,3,235000,NaN,13823.53
ag670426-342305226,6,a,2,134500,NaN,22416.67
mgc-220870264-pp-191014-150427,5,m,5,276935,NaN,55387.00
151800393,8,mn,4,216670,NaN,27083.75
146173003,2,an,43,159000,NaN,79500.00


In [386]:
# duree_int
seuil_min = 0
seuil_max = 500
col = 'duree_int'
print("\n---------------------- duree int ---------------------")
print(f"duree int minimal                      : {sales_df[col].min()}")
print(f"duree int médian                       : {sales_df[col].median()}")
print(f"duree int maximal                      : {sales_df[col].max()}")
print(f"duree int < {seuil_min}                          : {len(sales_df.loc[sales_df[col] < seuil_min])}")
print(f"duree int > {seuil_max}                        : {len(sales_df.loc[sales_df[col] > seuil_max])}")
print("-" * 55)
print("Ce type de champ est difficile à interpréter. Il peut s'agir d'un champ calculé et nous ne savons pas exactement quelle durée est prise en compte.")
print("\n")
sales_df[[col]].describe()


---------------------- duree int ---------------------
duree int minimal                      : -1125.0
duree int médian                       : 22.0
duree int maximal                      : 793.0
duree int < 0                          : 5159
duree int > 500                        : 60
-------------------------------------------------------
Ce type de champ est difficile à interpréter. Il peut s'agir d'un champ calculé et nous ne savons pas exactement quelle durée est prise en compte.




,duree_int
count,26196.000000
mean,-118.181898
std,390.366391
min,-1125.000000
25%,6.000000
50%,22.000000
75%,77.000000
max,793.000000


> **Recherche informations sur les valeurs manquantes**

In [54]:
# Surface terrain
sales_df[(sales_df['surface_terrain'].isna()) & (sales_df['prix_terrain'].isna() == False)]

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
158581267,pr,mn,v,0,100,NaN,4,236050,236049.0,1.0,0,0,0,0,0,NaN,47.81025,7.31556,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-05,68376,102,683760102,H,H,6837601,68701,44,68,NaN,NaN,NaN,10.93,4.0,0.056,2360.5


In [82]:
# Prix maison
sales_df[(sales_df['prix_maison'].isna()) & (sales_df['typedebien'].isin(['a', 'an']))].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
immo-facile-52477215,pr,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,NaN,NaN,True,0,G,NaN,1.0,1.0,NaN,NaN,NaN,False,2.0,180.0,Fioul,Radiateur,Collectif,a,n,8.0,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,7.93,3.0,0.092,7.93,3.0,0.092,1039.47
ag751938-380728492,pr,a,v,5,82,NaN,4,160000,NaN,NaN,0,1,0,0,C,167.0,47.74310,7.33110,Oui,7.0,NaN,NaN,True,Nord-Ouest,C,NaN,NaN,NaN,NaN,NaN,NaN,True,60.0,2400.0,NaN,NaN,NaN,network,n,8.0,a,2023-04,68224,201,682240201,H,H,6822401,68701,44,68,9.56,12.0,0.059,9.22,388.0,0.057,1951.22
ag681158-381115288,pr,a,v,0,69,NaN,3,89500,NaN,NaN,0,0,1,1,D,212.0,47.74453,7.33178,Oui,NaN,NaN,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-04,68224,201,682240201,H,H,6822401,68701,44,68,9.99,19.0,0.092,10.00,839.0,0.093,1297.10
ag751938-381003850,pr,a,v,8,154,NaN,4,240000,NaN,NaN,0,0,0,1,E,297.0,47.74539,7.31770,Non,8.0,NaN,1.0,True,0,E,NaN,NaN,NaN,NaN,NaN,NaN,True,39.0,5448.0,Gaz,NaN,Individuel,network,n,8.0,a,2023-04,68224,1101,682241101,H,H,6822401,68701,44,68,10.14,11.0,0.078,9.22,388.0,0.071,1558.44


In [92]:
# prix_terrain
sales_df[(sales_df['prix_terrain'].isna()) & (sales_df['surface_terrain'].isna())].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
immo-facile-52477215,pr,a,v,0,76,NaN,4,79000,NaN,NaN,0,0,1,0,G,379.0,48.17061,7.11870,Oui,1.0,NaN,NaN,True,0,G,NaN,1.0,1.0,NaN,NaN,NaN,False,2.0,180.0,Fioul,Radiateur,Collectif,a,n,8.0,a,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,7.93,3.0,0.092,7.93,3.0,0.092,1039.47
ag751938-380728492,pr,a,v,5,82,NaN,4,160000,NaN,NaN,0,1,0,0,C,167.0,47.74310,7.33110,Oui,7.0,NaN,NaN,True,Nord-Ouest,C,NaN,NaN,NaN,NaN,NaN,NaN,True,60.0,2400.0,NaN,NaN,NaN,network,n,8.0,a,2023-04,68224,201,682240201,H,H,6822401,68701,44,68,9.56,12.0,0.059,9.22,388.0,0.057,1951.22
ag681158-381115288,pr,a,v,0,69,NaN,3,89500,NaN,NaN,0,0,1,1,D,212.0,47.74453,7.33178,Oui,NaN,NaN,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-04,68224,201,682240201,H,H,6822401,68701,44,68,9.99,19.0,0.092,10.00,839.0,0.093,1297.10
ag751938-381003850,pr,a,v,8,154,NaN,4,240000,NaN,NaN,0,0,0,1,E,297.0,47.74539,7.31770,Non,8.0,NaN,1.0,True,0,E,NaN,NaN,NaN,NaN,NaN,NaN,True,39.0,5448.0,Gaz,NaN,Individuel,network,n,8.0,a,2023-04,68224,1101,682241101,H,H,6822401,68701,44,68,10.14,11.0,0.078,9.22,388.0,0.071,1558.44


In [100]:
# dpeC
sales_df[(sales_df['dpeC'].isna()) & (sales_df['dpeL'].isin(['0', 'NS', 'VI']))].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22
ag681104-381823822,pr,m,v,0,90,872.0,4,170000,NaN,NaN,0,1,0,1,NS,NaN,47.54152,7.28713,Non,NaN,NaN,1.0,True,Ouest,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,13.0,m,2023-04,68273,0,682730000,Z,Z,6827300,68000,44,68,9.20,1.0,0.058,9.20,1.0,0.058,1888.89
entities-1042950-3869684,pr,m,v,0,134,3797.0,4,413500,NaN,NaN,0,0,0,1,0,NaN,48.02501,7.08173,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,o,20.0,m,2023-04,68223,0,682230000,Z,Z,6822300,68302,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3085.82
entities-1042950-3869211,pr,m,v,0,114,571.0,4,353363,NaN,NaN,0,0,0,1,0,NaN,48.02501,7.08173,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,o,20.0,m,2023-04,68223,0,682230000,Z,Z,6822300,68302,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3099.68


In [112]:
# nb_etages
sales_df[(sales_df['nb_etages'].isna()) & (sales_df['etage'] == 0)].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
immo-facile-52471952,pr,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,1512.83
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22
ag681104-381823822,pr,m,v,0,90,872.0,4,170000,NaN,NaN,0,1,0,1,NS,NaN,47.54152,7.28713,Non,NaN,NaN,1.0,True,Ouest,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,13.0,m,2023-04,68273,0,682730000,Z,Z,6827300,68000,44,68,9.20,1.0,0.058,9.20,1.0,0.058,1888.89
ag681158-383546936,pr,m,v,0,82,11140.0,4,295000,NaN,NaN,0,0,0,2,E,329.0,47.91827,7.13488,Oui,NaN,NaN,3.0,NaN,0,B,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68229,0,682290000,Z,Z,6822900,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3597.56


In [134]:
# parking
sales_df['parking'].isna().sum() / sales_df.shape[0] * 100 

99.90265710062371

In [158]:
# places_parking
sales_df['places_parking'].isna().sum() / sales_df.shape[0] * 100 

63.33777986083571

In [176]:
# cave
sales_df['cave'].value_counts()

cave
True     8626
False    4621
Name: count, dtype: int64

In [198]:
# ges_class
sales_df[(sales_df['ges_class'].isna()) & (sales_df['dpeC'].isna())].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22
ag681104-381823822,pr,m,v,0,90,872.0,4,170000,NaN,NaN,0,1,0,1,NS,NaN,47.54152,7.28713,Non,NaN,NaN,1.0,True,Ouest,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,13.0,m,2023-04,68273,0,682730000,Z,Z,6827300,68000,44,68,9.20,1.0,0.058,9.20,1.0,0.058,1888.89
entities-1042950-3869684,pr,m,v,0,134,3797.0,4,413500,NaN,NaN,0,0,0,1,0,NaN,48.02501,7.08173,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,o,20.0,m,2023-04,68223,0,682230000,Z,Z,6822300,68302,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3085.82
entities-1042950-3869211,pr,m,v,0,114,571.0,4,353363,NaN,NaN,0,0,0,1,0,NaN,48.02501,7.08173,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,o,20.0,m,2023-04,68223,0,682230000,Z,Z,6822300,68302,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3099.68


In [212]:
# annee_construction
sales_df.groupby('GRD_QUART')['annee_construction'].median()

GRD_QUART
6800100    1980.0
6800200    1969.0
6800401    1967.0
6800500    1986.0
6800600    2007.0
            ...  
6838200    2012.5
6838300    1884.5
6838400    1985.0
6838500    1974.5
6838600    1983.0
Name: annee_construction, Length: 366, dtype: float64

In [220]:
# nb_toilettes
sales_df['nb_toilettes'].value_counts()

nb_toilettes
1.0     9298
2.0     5067
3.0      837
4.0      125
6.0       54
5.0       31
0.0       13
11.0       2
7.0        2
9.0        2
10.0       2
15.0       1
8.0        1
13.0       1
Name: count, dtype: int64

In [232]:
# nb_terraces
sales_df[sales_df['nb_terraces'].isna()].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
hektor-robischung-2862,pr,a,v,0,64,NaN,3,128400,NaN,NaN,0,1,0,1,C,118.0,47.55424,7.34766,Non,1.0,NaN,NaN,False,0,C,2010.0,1.0,NaN,NaN,NaN,NaN,False,21.0,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68325,0,683250000,Z,Z,6832500,68113,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2006.25
apimo-82452515,pr,m,v,0,85,4950.0,4,189000,NaN,NaN,0,0,0,0,0,NaN,48.17073,7.10498,0,NaN,NaN,NaN,NaN,Sud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,2223.53
immo-facile-52471952,pr,m,v,0,187,3182.0,9,282900,NaN,NaN,0,0,0,0,E,251.0,48.16989,7.13037,Oui,NaN,NaN,NaN,NaN,0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68044,0,680440000,Z,Z,6804400,68000,44,68,NaN,NaN,NaN,NaN,NaN,NaN,1512.83
ag681088-383601401,pr,m,v,0,90,3071.0,5,317000,NaN,NaN,0,0,0,1,NS,NaN,47.88536,7.19668,Non,NaN,NaN,2.0,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68,9.77,2.0,0.033,9.77,2.0,0.033,3522.22
ag681104-381823822,pr,m,v,0,90,872.0,4,170000,NaN,NaN,0,1,0,1,NS,NaN,47.54152,7.28713,Non,NaN,NaN,1.0,True,Ouest,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,13.0,m,2023-04,68273,0,682730000,Z,Z,6827300,68000,44,68,9.20,1.0,0.058,9.20,1.0,0.058,1888.89


In [238]:
# nb_terraces
sales_df['surface_balcon'].isna().sum() / sales_df.shape[0] * 100 

97.62771748927426

In [254]:
# ascenseur
sales_df[(sales_df['ascenseur'].isna()) & (sales_df['nb_etages'] > 7)].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ag681109-381414703,pr,a,v,6,104,NaN,5,296400,NaN,NaN,0,1,0,1,E,264.0,47.58605,7.56158,Non,8.0,NaN,NaN,NaN,0,D,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-04,68297,104,682970104,H,H,6829701,68403,44,68,10.25,10.0,0.043,10.56,16.0,0.044,2850.00
guy-hoquet-immo-facile-6230010,pr,a,v,0,86,NaN,5,110200,NaN,NaN,0,0,0,0,E,NaN,47.75469,7.30983,Oui,9.0,NaN,NaN,NaN,0,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,364.0,3422.1,NaN,NaN,NaN,a,n,36.0,a,2023-03,68224,1801,682241801,H,H,6822401,68701,44,68,10.35,1.0,0.097,8.57,110.0,0.080,1281.40
guy-hoquet-immo-facile-6235307,pr,a,v,0,184,NaN,7,430000,NaN,NaN,0,0,2,2,C,117.0,47.74915,7.33508,Oui,10.0,NaN,NaN,NaN,0,C,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,8184.0,NaN,NaN,NaN,a,n,15.0,a,2023-03,68224,101,682240101,H,H,6822401,68701,44,68,9.26,3.0,0.048,8.72,13.0,0.045,2336.96
hektor-MAYAIMMO2020-2527,pr,a,v,4,71,NaN,4,70000,NaN,NaN,0,0,0,1,E,238.0,47.75989,7.32755,Oui,10.0,NaN,NaN,True,Nord-Sud,E,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,71.0,a,2023-02,68224,1601,682241601,H,H,6822401,68701,44,68,8.75,5.0,0.106,9.22,388.0,0.112,985.92
immo-facile-52043762,pr,a,v,6,42,NaN,2,221900,NaN,NaN,0,0,0,0,NS,NaN,47.58574,7.57322,Non,17.0,NaN,NaN,NaN,0,VI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-02,68297,104,682970104,H,H,6829701,68403,44,68,14.23,212.0,0.032,14.00,472.0,0.032,5283.33


In [290]:
# logement_neuf
sales_df[sales_df['logement_neuf'].isna() & (sales_df['typedebien'].isin(['an', 'mn']))].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,loyer_m2_median_n6,nb_log_n6,taux_rendement_n6,loyer_m2_median_n7,nb_log_n7,taux_rendement_n7,prix_m2_vente
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
160967197,pr,mn,v,0,100,400.0,5,262100,NaN,NaN,0,0,0,0,0,NaN,47.69281,7.32542,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-07,68093,0,680930000,Z,Z,6809300,68120,44,68,13.68,1.0,0.063,13.68,1.0,0.063,2621.00
160967869,pr,mn,v,0,98,400.0,5,275100,NaN,NaN,0,0,0,0,0,NaN,47.69281,7.32542,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-07,68093,0,680930000,Z,Z,6809300,68120,44,68,13.68,1.0,0.058,13.68,1.0,0.058,2807.14
160967707,pr,mn,v,0,133,700.0,5,378784,NaN,NaN,0,0,1,0,0,NaN,47.69281,7.32542,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-07,68093,0,680930000,Z,Z,6809300,68120,44,68,13.68,1.0,0.058,13.68,1.0,0.058,2848.00
161439097,pr,mn,v,0,95,428.0,5,289568,NaN,NaN,0,0,0,0,0,NaN,47.69281,7.32542,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-07,68093,0,680930000,Z,Z,6809300,68120,44,68,13.68,1.0,0.054,13.68,1.0,0.054,3048.08
160967705,pr,mn,v,0,103,700.0,4,343428,NaN,NaN,0,0,1,0,0,NaN,47.69281,7.32542,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cm,NaN,0.0,m,2020-07,68093,0,680930000,Z,Z,6809300,68120,44,68,NaN,NaN,NaN,NaN,NaN,NaN,3334.25


---

### **Analyse du dataset sur les locations immobilières**

---

In [4]:
url_rentals = 'https://raw.githubusercontent.com/klopstock-dviz/immo_vis/master/data/ech_annonces_locations_68.csv'

rentals_df = pd.read_csv(url_rentals, sep=';', index_col="idannonce")

display(rentals_df.head())

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
immo-facile-52417007,pr,m,l,0,116,NaN,5,1250,NaN,NaN,0,0,0,1,D,226.0,47.88299,7.19650,0,1.0,NaN,NaN,True,0,B,1987.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,Radiateur,Individuel,a,n,15.0,m,2023-04,68159,0,681590000,Z,Z,6815900,68000,44,68
ag672097-382986683,pr,a,l,3,44,NaN,3,600,NaN,NaN,0,0,1,0,D,220.0,47.75081,7.32245,0,3.0,NaN,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Électrique,NaN,Individuel,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag672097-383464082,pr,a,l,1,53,NaN,2,498,NaN,NaN,0,1,1,0,E,272.0,47.75094,7.32239,0,NaN,NaN,NaN,NaN,0,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag680864-383044166,pr,a,l,0,41,NaN,2,460,NaN,NaN,0,0,1,1,VI,NaN,47.75204,7.32135,0,4.0,NaN,NaN,False,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,NaN,NaN,Gaz,Radiateur,Collectif,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag672097-380998252,pr,a,l,2,74,NaN,4,695,NaN,NaN,0,0,0,1,D,206.0,47.75075,7.32253,0,3.0,NaN,NaN,NaN,0,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaz,NaN,Individuel,a,n,20.0,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68


In [43]:
dataset_info = pd.DataFrame({
    'Value': [rentals_df.shape[0], 
              rentals_df.shape[1], 
              rentals_df.duplicated().sum(), 
              round((rentals_df.isna().sum().sum() / rentals_df.size) * 100, 2),
              len(rentals_df.select_dtypes('number').columns),
              len(rentals_df.select_dtypes('object').columns)
             ]
}, index=['nombre de lignes', 
          'nombre de colonnes', 
          'nombre de doublons', 
          'manquants (%)', 
          'nombre de variables quantitatives', 
          'nombre de variables qualitatives'])
dataset_info

,Value
nombre de lignes,11702.00
nombre de colonnes,51.00
nombre de doublons,0.00
manquants (%),29.69
nombre de variables quantitatives,31.00
nombre de variables qualitatives,20.00


In [45]:
dtype_info = pd.DataFrame({
    'colonne': rentals_df.columns,
    'type': rentals_df.dtypes
}).set_index('colonne')

dtype_info

,type
colonne,
type_annonceur,object
typedebien,object
typedetransaction,object
etage,int64
surface,int64
surface_terrain,float64
nb_pieces,int64
prix_bien,int64
prix_maison,float64


In [51]:
missing_percent = (rentals_df.isnull().sum() / len(rentals_df)) * 100

missing_df = pd.DataFrame({
    'colonne': rentals_df.columns,
    'manquants (%)': round(missing_percent,2)
}).set_index('colonne')

display(missing_df.sort_values(by='manquants (%)', ascending=False))

,manquants (%)
colonne,
charges_copro,100.00
parking,100.00
prix_maison,100.00
prix_terrain,100.00
nb_logements_copro,99.92
surface_balcon,98.35
surface_terrain,95.67
videophone,95.64
nb_terraces,86.63


> **Modalités des valeurs qualitatives**

In [13]:
cat_columns = rentals_df.select_dtypes('object')
cat_modes = []
for col in cat_columns:
    modalities = ', '.join(rentals_df[col].unique().astype(str))
    cat_modes.append({'colonne' : col, 'modalités': modalities, 'nombre modalités' : rentals_df[col].nunique()})

modalities_df = pd.DataFrame(cat_modes).set_index('colonne')
modalities_df

,modalités,nombre modalités
colonne,,
type_annonceur,"pr, pa",2
typedebien,"m, a, l",3
typedetransaction,"l, lt",2
dpeL,"D, E, VI, C, 0, B, G, NS, F, A",10
annonce_exclusive,"0, Non, Oui",3
cave,"True, nan, False",2
exposition,"0, sud-ouest, Est, Sud, Ouest, Nord-Ouest, Sud...",25
ges_class,"B, E, nan, D, F, C, G, A, VI, NS",9
videophone,"nan, True, False",2


> **Modalités des valeurs quantitatives**

In [136]:
num_columns = rentals_df.select_dtypes('number')
num_modes = []
for col in num_columns:
    if len(rentals_df[col].unique()) < 10:
        modalities = ', '.join(rentals_df[col].unique().astype(str))
        num_modes.append({'colonne' : col, 'modalités': modalities})

modalities_df = pd.DataFrame(num_modes).set_index('colonne')
modalities_df

,modalités
colonne,
prix_maison,nan
prix_terrain,nan
mensualiteFinance,0
balcon,"0, 1, 2, 3, 4, 7, 5, 12, 10"
eau,"0, 1, 2, 3"
bain,"1, 0, 2, 127, 4, 8, 7, 3, 5"
parking,nan
nb_terraces,"nan, 0.0, 2.0, 1.0, 3.0"
nb_logements_copro,"nan, 3.0, 8.0, 5.0, 0.0, 131.0, 10.0"


> **Répartition temporelle**

In [19]:
rentals_df['year'] = pd.to_datetime(rentals_df['date']).dt.year
rentals_df.groupby('year').size()

year
2019     373
2020    3733
2021    2855
2022    3533
2023    1208
dtype: int64

> **Recherches informations outliers**

In [33]:
# Etage
col = 'etage'
print("\n---------------------- Etage ---------------------")
print(f"Etage minimal                         : {rentals_df[col].min()}")
print(f"Nombre etage minimal                  : {len(rentals_df.loc[rentals_df[col] == rentals_df[col].min()])}")
print(f"Etage maximal                         : {rentals_df[col].max()}")
print(f"Nombre etage maximal                  : {len(rentals_df.loc[rentals_df[col] == rentals_df[col].max()])}")
print("-" * 50)
print("Ces valeurs ne semblent concernés que les appartements")
print("Pour les etages -1 une hypothèse pour des souplex ou sous-sols aménagés, la décote varie entre 15 et 30 % selon l’emplacement")
print("Pour les etages 999 ces biens semblent être regroupés dans des zones géographiques précises")
print("\n")
display(rentals_df.loc[rentals_df[col] == rentals_df[col].min(), [
    'surface', 'typedebien', 'balcon', 'cave', col]])

display(rentals_df.loc[rentals_df[col] == rentals_df[col].max(), [
    'surface', 'typedebien', 'GRD_QUART', 'mapCoordonneesLatitude', 'mapCoordonneesLongitude']])

rentals_df.loc[rentals_df[col] == rentals_df[col].min()]


---------------------- Etage ---------------------
Etage minimal                         : -2
Nombre etage minimal                  : 2
Etage maximal                         : 999
Nombre etage maximal                  : 7
--------------------------------------------------
Ces valeurs ne semblent concernés que les appartements
Pour les etages -1 une hypothèse pour des souplex ou sous-sols aménagés, la décote varie entre 15 et 30 % selon l’emplacement
Pour les etages 999 ces biens semblent être regroupés dans des zones géographiques précises




,surface,typedebien,balcon,cave,etage
idannonce,,,,,
ag681156-381174439,54,a,0,NaN,-2
ag681156-299082243,54,a,0,NaN,-2


,surface,typedebien,GRD_QUART,mapCoordonneesLatitude,mapCoordonneesLongitude
idannonce,,,,,
apimo-7998597,74,a,6822401,47.75877,7.35113
apimo-7605713,112,a,6812600,47.56247,7.52617
apimo-7419796,102,a,6829701,47.58605,7.56158
apimo-7338443,44,a,6829701,47.58605,7.56158
apimo-6743402,36,a,6829701,47.58627,7.56049
apimo-6504880,94,a,6814901,47.59415,7.57946
apimo-6356926,54,a,6829701,47.58605,7.56158


,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,...,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP,year
idannonce,,,,,,,,,,,,,,,,,,,,,
ag681156-381174439,pr,a,l,-2,54,NaN,3,880,NaN,NaN,...,68362,0,683620000,Z,Z,6836200,68000,44,68,2023
ag681156-299082243,pr,a,l,-2,54,NaN,3,910,NaN,NaN,...,68362,0,683620000,Z,Z,6836200,68000,44,68,2021


> **Recherche informations sur les valeurs manquantes**

In [51]:
# prix_maison
print(len(rentals_df[rentals_df['prix_maison'].notna()]))
# prix_terrain
print(len(rentals_df[rentals_df['prix_terrain'].notna()]))
# parking
print(len(rentals_df[rentals_df['parking'].notna()]))
# charges_copro
print(len(rentals_df[rentals_df['charges_copro'].notna()]))

0
0
0
0


In [47]:
# surface_terrain
rentals_df['surface_terrain'].isna().sum() / len(rentals_df) * 100

95.6674072808067

In [61]:
# dpeC
rentals_df[(rentals_df['dpeC'].isna()) & (rentals_df['dpeL'].isin(['0', 'NS', 'VI']))].head()

,type_annonceur,typedebien,typedetransaction,etage,surface,surface_terrain,nb_pieces,prix_bien,prix_maison,prix_terrain,mensualiteFinance,balcon,eau,bain,dpeL,dpeC,mapCoordonneesLatitude,mapCoordonneesLongitude,annonce_exclusive,nb_etages,parking,places_parking,cave,exposition,ges_class,annee_construction,nb_toilettes,nb_terraces,videophone,porte_digicode,surface_balcon,ascenseur,nb_logements_copro,charges_copro,chauffage_energie,chauffage_systeme,chauffage_mode,categorie_annonceur,logement_neuf,duree_int,typedebien_lite,date,INSEE_COM,IRIS,CODE_IRIS,TYP_IRIS_x,TYP_IRIS_y,GRD_QUART,UU2010,REG,DEP
idannonce,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ag680864-383044166,pr,a,l,0,41,NaN,2,460,NaN,NaN,0,0,1,1,VI,NaN,47.75204,7.32135,0,4.0,NaN,NaN,False,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,NaN,NaN,Gaz,Radiateur,Collectif,a,n,NaN,a,2023-04,68224,1201,682241201,H,H,6822401,68701,44,68
ag753633-380751028,pr,a,l,3,102,NaN,3,900,NaN,NaN,0,0,0,1,VI,NaN,47.74146,7.34618,0,NaN,NaN,NaN,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,Collectif,a,n,8.0,a,2023-04,68224,701,682240701,H,H,6822401,68701,44,68
rodacom-5269984,pr,a,l,1,62,NaN,3,805,NaN,NaN,0,0,0,0,0,NaN,47.75847,7.34729,0,NaN,NaN,NaN,False,0,NaN,2016.0,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-04,68224,401,682240401,H,H,6822401,68701,44,68
ag681106-383371190,pr,a,l,2,78,NaN,3,800,NaN,NaN,0,0,1,0,NS,NaN,47.74381,7.33655,0,2.0,NaN,2.0,False,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,a,n,NaN,a,2023-04,68224,102,682240102,H,H,6822401,68701,44,68
ag681088-383462309,pr,a,l,2,70,NaN,3,740,NaN,NaN,0,1,0,1,NS,NaN,47.74942,7.33994,0,3.0,NaN,NaN,True,0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gaz,NaN,Individuel,a,n,NaN,a,2023-04,68224,102,682240102,H,H,6822401,68701,44,68


In [73]:
# nb_etages
rentals_df[(rentals_df['nb_etages'].isna()) & (rentals_df['etage'] == 0)][['etage', 'surface', 'nb_etages', 'nb_pieces']].head()

,etage,surface,nb_etages,nb_pieces
idannonce,,,,
hektor-agfreyimmo-760,0,70,NaN,2
ag681119-383455202,0,65,NaN,3
guy-hoquet-immo-facile-6333045,0,100,NaN,4
ag680864-383044196,0,41,NaN,3
ag680896-382386169,0,59,NaN,2


In [77]:
# ges_class
rentals_df[(rentals_df['ges_class'].isna()) & (rentals_df['dpeC'].isna())][['dpeC', 'dpeL', 'ges_class', 'typedebien']].head()

,dpeC,dpeL,ges_class,typedebien
idannonce,,,,
ag680864-383044166,NaN,VI,NaN,a
ag753633-380751028,NaN,VI,NaN,a
rodacom-5269984,NaN,0,NaN,a
ag681106-383371190,NaN,NS,NaN,a
ag681088-383462309,NaN,NS,NaN,a


In [79]:
# cave
rentals_df['cave'].value_counts()

cave
True     2910
False    2087
Name: count, dtype: int64

In [93]:
# annee_construction
rentals_df.groupby('GRD_QUART')[['annee_construction']].median().head()
print(rentals_df['annee_construction'].isna().sum() / len(rentals_df) * 100)

80.55887882413263


In [105]:
# nb_toilettes
rentals_df['nb_toilettes'].value_counts()

nb_toilettes
1.0     4205
2.0      535
3.0       28
0.0       15
4.0        3
6.0        2
12.0       1
14.0       1
5.0        1
Name: count, dtype: int64

In [109]:
# ascenseur
rentals_df[(rentals_df['ascenseur'].isna()) & (rentals_df['nb_etages'] > 7)][['etage', 'nb_etages', 'ascenseur']].head()

,etage,nb_etages,ascenseur
idannonce,,,
immo-facile-51524190,12,12.0,NaN
orpi-1-054039E2INSF,0,10.0,NaN
ag680967-363304816,15,15.0,NaN
immo-facile-49832225,3,12.0,NaN
ag681156-343448364,5,8.0,NaN


In [119]:
# chauffage_energie
rentals_df[(rentals_df['chauffage_energie'].isna()) & 
           (rentals_df['chauffage_systeme'].notna())][['chauffage_energie', 'chauffage_systeme', 'chauffage_mode']].head()

,chauffage_energie,chauffage_systeme,chauffage_mode
idannonce,,,
citya-immobilier-1273-GES89910117-63,NaN,Sol,Individuel
hektor-expertiseimmo-2919,NaN,Radiateur,NaN
immo-facile-52555775,NaN,Radiateur,NaN
ag681494-383189717,NaN,Radiateur,Collectif
ag681494-382194877,NaN,Radiateur,Collectif


In [123]:
# chauffage_energie
rentals_df.groupby('chauffage_systeme')[['chauffage_energie', 'chauffage_mode']].first()

,chauffage_energie,chauffage_mode
chauffage_systeme,,
Climatisation révérsible,Électrique,None
Convecteur,Électrique,Individuel
Pompe à chaleur,None,Individuel
"Pompe à chaleur, Radiateur",None,Individuel
"Pompe à chaleur, Sol",None,Individuel
Poêle à Bois,Bois,None
Radiateur,Électrique,Individuel
Sol,Gaz,Collectif
"Sol, Radiateur",Électrique,Individuel


In [149]:
# logement_neuf
print(len(rentals_df[rentals_df['logement_neuf'].isna() & (rentals_df['typedebien'].isin(['an', 'mn']))]))
rentals_df[(rentals_df['logement_neuf'].notna()) & (rentals_df['logement_neuf'] != 'n')][['logement_neuf']]

0


,logement_neuf
idannonce,
gedeon-26584051,o


In [169]:
# duree_int
rentals_df[(rentals_df['duree_int'].notna()) & (rentals_df['duree_int'] < 0)][['duree_int']].head()

,duree_int
idannonce,
guy-hoquet-immo-facile-4745406,-1104.0
hektor-deuxmm-74,-1118.0
ag680145-284675234,-1111.0
ag680145-284675306,-1111.0
guy-hoquet-immo-facile-4740067,-1104.0


---

## Analyse du dataset valeurs foncières - Départements

---

In [5]:
# Exemple pour avoir un aperçu
year = "2024"
num_dep = "01"
url = f"https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/departements/{num_dep}.csv.gz"

df_dvf = pd.read_csv(url, compression='gzip', low_memory=False, index_col=0)
df_dvf.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
id_mutation,,,,,,,,,,,,,,,,,,,,,
2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,1076,...,NaN,NaN,NaN,P,prés,NaN,NaN,99.0,5.530952,45.952439
2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,1103,...,NaN,NaN,NaN,S,sols,NaN,NaN,115.0,6.043339,46.282256
2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,1203,...,NaN,NaN,NaN,S,sols,NaN,NaN,497.0,4.911143,46.247235
2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,1173,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.058695,46.332212
2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,1173,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.058695,46.332212


In [68]:
# Exemple pour avoir un aperçu
year = "2024"
num_dep = "01"
num_commune = "01001"
url = f"https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/communes/{num_dep}/{num_commune}.csv"

df_com = pd.read_csv(url, index_col=0)
df_com.head()

,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
id_mutation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-1441,2024-02-20,1,Vente,2070.6,NaN,NaN,POULIN,B043,1400,1001,L'Abergement-Clémenciat,1,NaN,NaN,010010000B0248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,BS,taillis sous futaie,NaN,NaN,13804,4.935807,46.146202
2024-1720,2024-03-04,3,Echange,10013.5,NaN,NaN,SUR BRODY,B005,1400,1001,L'Abergement-Clémenciat,1,NaN,NaN,01001000ZA0038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,4640,4.909367,46.171004
2024-1720,2024-03-04,3,Echange,10013.5,NaN,NaN,SUR BRODY,B005,1400,1001,L'Abergement-Clémenciat,1,NaN,NaN,01001000ZA0038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,23970,4.909367,46.171004
2024-4311,2024-05-16,1,Vente,230000.0,132.0,NaN,IMP DES CHENES,0100,1400,1001,L'Abergement-Clémenciat,1,NaN,NaN,010010000A0952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,116.0,6.0,S,sols,NaN,NaN,988,4.922756,46.149338
2024-4724,2024-05-30,1,Vente,292300.0,76.0,NaN,IMP BELLEVUE,0040,1400,1001,L'Abergement-Clémenciat,1,NaN,NaN,01001000ZL0223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,91.0,4.0,S,sols,NaN,NaN,500,4.924485,46.134200


> Récupération de tous les datasets full.csv.gz par année

<div class="alert alert-warning">
    <i class="fa fa-info-circle"></i> Le processus est long et prend plusieurs minutes
</div>

In [41]:
years = ["2019", "2020", "2021", "2022", "2023", "2024"]
base_url_template = "https://files.data.gouv.fr/geo-dvf/latest/csv/{year}/"

def fetch_and_process_file(link):
    """Télécharge et lit un fichier GZ, retourne un DataFrame."""
    try:
        file_response = requests.get(link)
        file_response.raise_for_status()
        with gzip.open(BytesIO(file_response.content), 'rt', encoding='utf-8') as f:
            return pd.read_csv(f, low_memory=False)
    except Exception as e:
        print(f"Erreur avec le fichier {link}: {e}")
        return pd.DataFrame()  # Retourne un DataFrame vide en cas d'erreur

def process_year(year):
    """Récupère tous les fichiers .gz pour une année spécifique."""
    base_url = base_url_template.format(year=year)
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [base_url + a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.gz')]
        print(f"Année {year}: {len(links)} fichier(s) trouvé(s).")
        
        # Télécharger et traiter les fichiers en parallèle
        with ThreadPoolExecutor() as executor:
            dfs = list(executor.map(fetch_and_process_file, links))
        
        # Concaténer tous les DataFrames pour l'année
        merged_df = pd.concat(dfs, ignore_index=True)
        merged_df.reset_index(drop=True, inplace=True)

        # Sauvegarder le DataFrame consolidé pour l'année
        output_path = f"../data/raw/full_{year}.csv.gz"
        merged_df.to_csv(output_path, index=False, compression='gzip', encoding='utf-8')
        print(f"Année {year}: Traitement terminé et sauvegardé dans {output_path}.")
    else:
        print(f"Erreur lors de l'accès à l'URL pour l'année {year}: {response.status_code}")

# Processus principal
for year in years:
    print(f"Traitement des fichiers pour l'année {year}...")
    process_year(year)

print("Tous les traitements sont terminés.")

Traitement des fichiers pour l'année 2019...
Année 2019: 1 fichiers trouvés.
Année 2019: Traitement terminé et sauvegardé dans ../data/raw/full_2019.csv.gz.
Traitement des fichiers pour l'année 2020...
Année 2020: 1 fichiers trouvés.
Année 2020: Traitement terminé et sauvegardé dans ../data/raw/full_2020.csv.gz.
Traitement des fichiers pour l'année 2021...
Année 2021: 1 fichiers trouvés.
Année 2021: Traitement terminé et sauvegardé dans ../data/raw/full_2021.csv.gz.
Traitement des fichiers pour l'année 2022...
Année 2022: 1 fichiers trouvés.
Année 2022: Traitement terminé et sauvegardé dans ../data/raw/full_2022.csv.gz.
Traitement des fichiers pour l'année 2023...
Année 2023: 1 fichiers trouvés.
Année 2023: Traitement terminé et sauvegardé dans ../data/raw/full_2023.csv.gz.
Traitement des fichiers pour l'année 2024...
Année 2024: 1 fichiers trouvés.
Année 2024: Traitement terminé et sauvegardé dans ../data/raw/full_2024.csv.gz.
Tous les traitements sont terminés.


In [51]:
output_path = f"../data/raw/full_years.csv.gz"

if os.path.exists(output_path):
    print("Le fichier existe déjà. Chargement dans un DataFrame...")
    with gzip.open(output_path, 'rt', encoding='utf-8') as f:
        full_merged_df = pd.read_csv(f, low_memory=False)
else:
    # Merge tous les datasets des années en 1 seul dataset
    years = ["2019", "2020", "2021", "2022", "2023", "2024"]
    datasets = ['full_'+year+'.csv.gz' for year in years]
    full_merged_df = pd.DataFrame()
    for dataset in datasets:
        with gzip.open('../data/raw/'+ dataset, 'rt', encoding='utf-8') as f:
            print("process ", dataset)
            temp_df = pd.read_csv(f, low_memory=False)
            full_merged_df = pd.concat([full_merged_df, temp_df])
            full_merged_df.reset_index(drop=True, inplace=True)
            del temp_df
        
    full_merged_df.to_csv(output_path, index=False, compression='gzip', encoding='utf-8')

Le fichier existe déjà. Chargement dans un DataFrame...


In [98]:
full_merged_df.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,ancien_code_commune,ancien_nom_commune,id_parcelle,ancien_id_parcelle,numero_volume,lot1_numero,lot1_surface_carrez,lot2_numero,lot2_surface_carrez,lot3_numero,lot3_surface_carrez,lot4_numero,lot4_surface_carrez,lot5_numero,lot5_surface_carrez,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2019-1,2019-07-01,1,Vente,244200.0,73.0,NaN,RUE COMMANDANT COUSTEAU,0958,1000.0,01053,Bourg-en-Bresse,01,NaN,NaN,01053000CN0405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,150.0,5.0,S,sols,NaN,NaN,695.0,5.264531,46.194443
1,2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,AG,terrains d'agrément,NaN,NaN,888.0,5.415689,46.159272
2,2019-2,2019-07-04,1,Vente,195000.0,852.0,NaN,CHE DE LA CROIX ROSIER,0170,1250.0,01184,Hautecourt-Romanèche,01,NaN,NaN,01184000AM0237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,94.0,4.0,S,sols,NaN,NaN,500.0,5.415689,46.159272
3,2019-3,2019-07-04,1,Vente,110030.0,5041.0,NaN,LA VILLE,B097,1250.0,01177,Grand-Corent,01,NaN,NaN,011770000B0764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,Maison,90.0,3.0,S,sols,NaN,NaN,522.0,5.431065,46.199271
4,2019-4,2019-07-03,1,Vente,160450.0,NaN,NaN,CHARBONNET,B039,1270.0,01108,Coligny,01,NaN,NaN,01108000AH0010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,S,sols,NaN,NaN,8.0,5.353051,46.384396


In [53]:
dataset_info = pd.DataFrame({
    'Value': [full_merged_df.shape[0], 
              full_merged_df.shape[1], 
              full_merged_df.duplicated().sum(), 
              round((full_merged_df.isna().sum().sum() / full_merged_df.size) * 100, 2),
              len(full_merged_df.select_dtypes('number').columns),
              len(full_merged_df.select_dtypes('object').columns)
             ]
}, index=['nombre de lignes', 
          'nombre de colonnes', 
          'nombre de doublons', 
          'manquants (%)', 
          'nombre de variables quantitatives', 
          'nombre de variables qualitatives'])
dataset_info

,Value
nombre de lignes,20195667.00
nombre de colonnes,40.00
nombre de doublons,1302023.00
manquants (%),49.06
nombre de variables quantitatives,17.00
nombre de variables qualitatives,23.00


In [55]:
dtype_info = pd.DataFrame({
    'colonne': full_merged_df.columns,
    'type': full_merged_df.dtypes
}).set_index('colonne')

dtype_info

,type
colonne,
id_mutation,object
date_mutation,object
numero_disposition,int64
nature_mutation,object
valeur_fonciere,float64
adresse_numero,float64
adresse_suffixe,object
adresse_nom_voie,object
adresse_code_voie,object


> **Valeurs manquantes**

In [20]:
missing_percent = (full_merged_df.isnull().sum() / len(full_merged_df)) * 100

missing_df = pd.DataFrame({
    'colonne': full_merged_df.columns,
    'manquants (%)': round(missing_percent,2)
}).set_index('colonne')

display(missing_df.sort_values(by='manquants (%)', ascending=False))

,manquants (%)
colonne,
ancien_id_parcelle,100.00
ancien_code_commune,99.99
ancien_nom_commune,99.99
lot5_surface_carrez,99.97
lot4_surface_carrez,99.93
lot5_numero,99.78
numero_volume,99.76
lot3_surface_carrez,99.70
lot4_numero,99.50


> **Valeurs qualitatives**

In [24]:
cat_columns = full_merged_df.select_dtypes('object')
cat_modes = []
for col in cat_columns:
    modalities = ', '.join(full_merged_df[col].unique().astype(str))
    cat_modes.append({'colonne' : col, 'modalités': modalities, 'nombre modalités': full_merged_df[col].nunique()})

modalities_df = pd.DataFrame(cat_modes).set_index('colonne')
modalities_df

,modalités,nombre modalités
colonne,,
id_mutation,"2019-1, 2019-2, 2019-3, 2019-4, 2019-5, 2019-6...",7622875
date_mutation,"2019-07-01, 2019-07-04, 2019-07-03, 2019-07-02...",1816
nature_mutation,"Vente, Echange, Vente terrain à bâtir, Vente e...",6
adresse_suffixe,"nan, A, B, C, 2, E, I, P, T, N, D, K, S, Z, G,...",40
adresse_nom_voie,"RUE COMMANDANT COUSTEAU, CHE DE LA CROIX ROSIE...",1258408
adresse_code_voie,"0958, 0170, B097, B039, B119, 0031, 0605, 3160...",20121
code_commune,"01053, 01184, 01177, 01108, 01451, 01374, 0130...",33375
nom_commune,"Bourg-en-Bresse, Hautecourt-Romanèche, Grand-C...",31167
code_departement,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12...",97


> **Valeurs quantitatives**

In [57]:
num_columns = full_merged_df.select_dtypes('number')
num_modes = []
for col in num_columns:
    modalities = ', '.join(full_merged_df[col].unique().astype(str))
    num_modes.append({'colonne' : col, 'modalités': modalities, 'nombre modalités': full_merged_df[col].nunique()})

modalities_df = pd.DataFrame(num_modes).set_index('colonne')
modalities_df

,modalités,nombre modalités
colonne,,
numero_disposition,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,...",1204
valeur_fonciere,"244200.0, 195000.0, 110030.0, 160450.0, 211500...",466306
adresse_numero,"73.0, 852.0, 5041.0, nan, 5515.0, 1259.0, 97.0...",9154
code_postal,"1000.0, 1250.0, 1270.0, 1440.0, 1160.0, 1310.0...",5870
ancien_code_commune,"nan, 1097.0, 2475.0, 2434.0, 11329.0, 14666.0,...",107
lot1_surface_carrez,"nan, 58.39, 55.31, 105.11, 67.74, 117.01, 58.0...",28347
lot2_surface_carrez,"nan, 86.75, 41.8, 37.76, 94.2, 69.73, 36.6, 15...",17797
lot3_surface_carrez,"nan, 67.3, 91.71, 89.19, 63.71, 73.01, 89.25, ...",12414
lot4_surface_carrez,"nan, 91.71, 63.0, 57.09, 73.1, 69.49, 170.38, ...",5746


> **Recherche sur les valeurs manquantes**

In [3]:
output_path = f"../data/raw/full_2024.csv"
dvf_df = pd.read_csv(output_path, low_memory=False)

In [27]:
# valeur_fonciere
dvf_df['valeur_fonciere'].isna().sum() / dvf_df.shape[0]

0.010885696358391797

In [37]:
# adresse_nom_voie
dvf_df[(dvf_df['adresse_nom_voie'].isna()) & (dvf_df['nature_mutation'] == "Vente en l'état futur d'achèvement")].head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
1368,2024-401,2024-01-24,1,Vente en l'état futur d'achèvement,162600.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
1848,2024-562,2024-01-30,1,Vente en l'état futur d'achèvement,190080.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
2209,2024-644,2024-02-05,1,Vente en l'état futur d'achèvement,187920.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
2723,2024-856,2024-02-02,1,Vente en l'état futur d'achèvement,301000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.789195,45.938345
2724,2024-856,2024-02-02,1,Vente en l'état futur d'achèvement,301000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.789195,45.938345


In [47]:
# adresse_code_voie
dvf_df[(dvf_df['adresse_code_voie'].isna()) & (dvf_df['nature_mutation'] == "Vente en l'état futur d'achèvement")].head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
1368,2024-401,2024-01-24,1,Vente en l'état futur d'achèvement,162600.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
1848,2024-562,2024-01-30,1,Vente en l'état futur d'achèvement,190080.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
2209,2024-644,2024-02-05,1,Vente en l'état futur d'achèvement,187920.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.842245,46.309241
2723,2024-856,2024-02-02,1,Vente en l'état futur d'achèvement,301000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.789195,45.938345
2724,2024-856,2024-02-02,1,Vente en l'état futur d'achèvement,301000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.789195,45.938345


In [67]:
# lot1_numero
dvf_df[(dvf_df['lot1_numero'].isna() == False) & (dvf_df['id_parcelle'] == '01173000AI0551')][['lot1_numero', 'id_parcelle']]

,lot1_numero,id_parcelle
3,175,01173000AI0551
4,175,01173000AI0551
5,50,01173000AI0551
6,175,01173000AI0551


In [83]:
# lot1_surface_carrez
dvf_df[dvf_df['lot1_surface_carrez'].notna()][['surface_reelle_bati', 'surface_terrain', 'lot1_surface_carrez', 'nombre_lots']].head()

,surface_reelle_bati,surface_terrain,lot1_surface_carrez,nombre_lots
117,84.0,NaN,81.46,1
125,86.0,NaN,85.82,1
130,45.0,NaN,45.67,1
138,40.0,NaN,39.11,1
186,67.0,NaN,67.27,1


In [131]:
# lot2_numero
len(dvf_df[(dvf_df['lot2_numero'].isna()) & (dvf_df['nombre_lots'] > 1)])

0

In [129]:
# lot2_surface_carrez
len(dvf_df[dvf_df['lot2_surface_carrez'].isna() & dvf_df['lot2_numero'].notna() & (dvf_df['nombre_lots'] > 1)])

102396

In [137]:
# lot3_numero
len(dvf_df[(dvf_df['lot3_numero'].isna()) & (dvf_df['nombre_lots'] > 2)])

0

In [139]:
# lot3_surface_carrez
len(dvf_df[dvf_df['lot3_surface_carrez'].isna() & dvf_df['lot3_numero'].notna() & (dvf_df['nombre_lots'] > 2)])

20097

In [141]:
# lot4_numero
len(dvf_df[(dvf_df['lot4_numero'].isna()) & (dvf_df['nombre_lots'] > 3)])

0

In [143]:
# lot4_surface_carrez
len(dvf_df[dvf_df['lot4_surface_carrez'].isna() & dvf_df['lot4_numero'].notna() & (dvf_df['nombre_lots'] > 3)])

6604

In [145]:
# lot5_numero
len(dvf_df[(dvf_df['lot5_numero'].isna()) & (dvf_df['nombre_lots'] > 4)])

0

In [147]:
# lot5_surface_carrez
len(dvf_df[dvf_df['lot5_surface_carrez'].isna() & dvf_df['lot5_numero'].notna() & (dvf_df['nombre_lots'] > 4)])

2697

In [163]:
# code_type_local
dvf_df[(dvf_df['code_type_local'].notna()) & (dvf_df['type_local'].notna())][['code_postal', 'code_type_local', 'code_commune', 'type_local']].head()

,code_postal,code_type_local,code_commune,type_local
3,1170.0,3.0,01173,Dépendance
4,1170.0,3.0,01173,Dépendance
5,1170.0,3.0,01173,Dépendance
6,1170.0,2.0,01173,Appartement
77,1150.0,3.0,01202,Dépendance


In [169]:
# type_local (même principe que code_type_local)
dvf_df['type_local'].unique()

array([nan, 'Dépendance', 'Appartement', 'Maison',
       'Local industriel. commercial ou assimilé'], dtype=object)

In [189]:
# surface_reelle_bati
dvf_df[(dvf_df['surface_reelle_bati'].isna()) & 
       (dvf_df['surface_terrain'].notna())][['lot1_surface_carrez', 'surface_reelle_bati', 'surface_terrain', 'nature_culture']].head()

,lot1_surface_carrez,surface_reelle_bati,surface_terrain,nature_culture
0,NaN,NaN,99.0,prés
1,NaN,NaN,115.0,sols
2,NaN,NaN,497.0,sols
7,NaN,NaN,1584.0,futaies résineuses
8,NaN,NaN,188.0,futaies résineuses


In [201]:
# nombre_pieces_principales
dvf_df[(dvf_df['nombre_pieces_principales'].isna()) & 
       (dvf_df['surface_terrain'].notna())][['nombre_pieces_principales', 'code_nature_culture', 'surface_terrain', 'nature_culture']].head()

,nombre_pieces_principales,code_nature_culture,surface_terrain,nature_culture
0,NaN,P,99.0,prés
1,NaN,S,115.0,sols
2,NaN,S,497.0,sols
7,NaN,BR,1584.0,futaies résineuses
8,NaN,BR,188.0,futaies résineuses


In [199]:
# code_nature_culture
dvf_df['code_nature_culture'].unique()

array(['P', 'S', nan, 'BR', 'BT', 'L', 'J', 'T', 'AG', 'PA', 'BS', 'AB',
       'VI', 'BM', 'E', 'BF', 'BP', 'CH', 'CA', 'VE', 'B', 'PP', 'PC',
       'BO', 'TP', 'LB', 'PH', 'PE'], dtype=object)

In [219]:
# code_nature_culture_speciale
print(len(dvf_df[(dvf_df['code_nature_culture_speciale'].isna()) & 
           (dvf_df['nature_culture_speciale'].notna())]
    [['code_nature_culture', 'code_nature_culture_speciale', 'nature_culture', 'nature_culture_speciale']]))

display(dvf_df['code_nature_culture_speciale'].unique())

0


array([nan, 'FRICH', 'PATUR', 'MARAI', 'FOSSE', 'SAPIN', 'CHEM', 'PARC',
       'JARD', 'ACACI', 'FER', 'POTAG', 'PIN', 'MARE', 'VAOC', 'ETANG',
       'VDQS', 'SPORT', 'CHTRU', 'CHAT', 'IMM', 'FLOR', 'RUE', 'CANAL',
       'SAULE', 'HORT', 'SERRE', 'HETRE', 'CHENE', 'BROUS', 'PEPIN',
       'BIEF', 'PACAG', 'VAGUE', 'CAMP', 'OLIVE', 'NOYER', 'ABRIC',
       'MARNA', 'CITRO', 'RAIS', 'CHVER', 'AMAND', 'EPICE', 'PECHE',
       'LAC', 'ROC', 'SABLE', 'BOUL', 'BALLA', 'RESIN', 'CERIS', 'EUCAL',
       'MAREC', 'BASS', 'LEGUM', 'CHASS', 'RIZ', 'DIGUE', 'SALIN',
       'CLAIR', 'VIVIE', 'SART', 'MOTTE', 'FOUG', 'JOUAL', 'FAMIL',
       'BRUY', 'PECH', 'INTEN', 'PASS', 'RESER', 'CASS', 'PATIS', 'AULN',
       'POMME', 'ABREU', 'HIST', 'CRESS', 'AEROD', 'MAQUI', 'CHLIE',
       'ORANG', 'CEDRA', 'ASPER', 'PIEDS', 'RUINE', 'ALLEE', 'PAFEU',
       'AIRE', 'LAGUN', 'GRAVE', 'DUNE', 'HUITR', 'RIVAG', 'PRUNE',
       'CIDRE', 'PLVEN', 'KIWIS', 'ARGIL', 'POIRE', 'VIGNE', 'HAIES',
       'MARAM', '

In [223]:
# surface_terrain
dvf_df[(dvf_df['surface_reelle_bati'].notna()) & 
       (dvf_df['surface_terrain'].isna())][['lot1_surface_carrez', 'surface_reelle_bati', 'surface_terrain', 'nature_culture', 'type_local']].head()

,lot1_surface_carrez,surface_reelle_bati,surface_terrain,nature_culture,type_local
6,NaN,89.0,NaN,NaN,Appartement
85,NaN,110.0,NaN,NaN,Maison
101,NaN,80.0,NaN,NaN,Appartement
106,NaN,48.0,NaN,NaN,Appartement
117,81.46,84.0,NaN,NaN,Maison


In [239]:
# longitude
dvf_df[(dvf_df['longitude'].isna())][['adresse_numero', 'adresse_nom_voie', 'nature_culture', 'longitude', 'latitude',]].head()

,adresse_numero,adresse_nom_voie,nature_culture,longitude,latitude
146,NaN,CORBASSIERE,landes,NaN,NaN
147,NaN,CORBASSIERE,sols,NaN,NaN
148,47.0,CHE DE FONTANETTE,sols,NaN,NaN
149,47.0,CHE DE FONTANETTE,sols,NaN,NaN
150,47.0,CHE DE FONTANETTE,sols,NaN,NaN


In [241]:
# latitude
dvf_df[(dvf_df['latitude'].isna())][['adresse_numero', 'adresse_nom_voie', 'nature_culture', 'longitude', 'latitude',]].head()

,adresse_numero,adresse_nom_voie,nature_culture,longitude,latitude
146,NaN,CORBASSIERE,landes,NaN,NaN
147,NaN,CORBASSIERE,sols,NaN,NaN
148,47.0,CHE DE FONTANETTE,sols,NaN,NaN
149,47.0,CHE DE FONTANETTE,sols,NaN,NaN
150,47.0,CHE DE FONTANETTE,sols,NaN,NaN
